In [173]:
import pandas as pd
import scipy
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from math import log2

from numba import jit, cuda

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from des import des_enc, random_bytes, group_bytes, des_enc_list

In [174]:
import struct

def convert_to_bytes(list):
    return struct.pack('B' * len(list), *list)

NUM_BYTES_KEY = 7
NUM_BYTES_MSG = 8

def generate_samples(msg, N):
    inputs = []
    keys = set()
    labels = []
    msg_bytes = convert_to_bytes(msg)
    
    for _ in range(N):
        key = random_bytes(NUM_BYTES_KEY)
        key_num = int.from_bytes(key)
        key.append(0)
        while key_num in keys:
            key = random_bytes(NUM_BYTES_KEY)
            key_num = int.from_bytes(key)
            key.append(0)
        
        key_bytes = convert_to_bytes(key)
        cipher_bytes = des_enc(key_bytes, msg_bytes)
        cipher = [x for x in cipher_bytes]
        #cipher_num = int.from_bytes(cipher)
        
        inputs.append(cipher)
        labels.append(group_bytes(key, NUM_BYTES_KEY))
        keys.add(key_num)
    
    return np.array(inputs), np.array(labels)

In [175]:
def estimate_key(cipher, num_byte, inputs, keys):
    model = DecisionTreeRegressor(min_samples_leaf=20)
    model.fit(inputs, keys[:, num_byte])
    
    return model.predict([cipher])[0]

In [176]:

#TEST_KEY = 20217796232662129
TEST_KEY_BYTES = random_bytes(7)
TEST_KEY_GROUP = group_bytes(TEST_KEY_BYTES, NUM_BYTES_KEY)

TEST_MSG = random_bytes(8)
TEST_CIPHER = des_enc_list(TEST_KEY_BYTES, TEST_MSG)

p = 1
avg = [2**15 for _ in range(3)]
avg.append(128)
for x in np.abs(np.array(avg) - np.array(TEST_KEY_GROUP)).tolist():
    p *= x
log2(p)

47.61857273515298

In [177]:
N_ROUNDS = 1
N_SAMPLES = 100000

def round(msg, cipher, key, num_byte, N_SAMPLES):
    inputs, labels = generate_samples(msg, N_SAMPLES)
    key_bar = math.floor(estimate_key(cipher, num_byte, inputs, labels))
    return key[num_byte] - key_bar

In [178]:
def test_work(msgs, ciphers, keys, num_byte, N_SAMPLES, N_ROUNDS):
    P = {}
    abs_d = []
    
    for msg, cipher, key in zip(msgs, ciphers, keys):
        d = round(msg, cipher, key, num_byte, N_SAMPLES)
        abs_d.append(abs(d))
        if d in P:
            P[d] += 1 / N_ROUNDS
        else:
            P[d] = 1 / N_ROUNDS
    abs_d = np.array(abs_d)
    return P, np.min(abs_d), np.max(abs_d), np.mean(abs_d)

def test_all(num_byte, N_ROUNDS, N_SAMPLES):
    df = pd.read_csv('DES_TRAIN.csv')
    msgs = df.iloc[:N_ROUNDS, :(NUM_BYTES_MSG)].values
    ciphers = df.iloc[:N_ROUNDS, NUM_BYTES_MSG:(2 * NUM_BYTES_MSG)].values
    keys = df.iloc[:N_ROUNDS, (2 * NUM_BYTES_MSG):-1].values
    keys = [group_bytes(key, NUM_BYTES_KEY) for key in keys]
    
    return test_work(msgs, ciphers, keys, num_byte, N_SAMPLES, N_ROUNDS)

# P, min_d, max_d, avg_d = test_all(0, N_ROUNDS, N_SAMPLES)
# print('Min distance: ', min_d)
# print('Max distance: ', max_d)
# print('Average distance: ', avg_d)
# print('Distribution: ', P)
# plt.bar(list(P.keys()), list(P.values()))

In [185]:
exp_mean = []
exp_us = []

for _ in range(2):

    TEST_KEY_BYTES = random_bytes(7)
    TEST_KEY_GROUP = group_bytes(TEST_KEY_BYTES, NUM_BYTES_KEY)

    TEST_MSG = random_bytes(8)
    TEST_CIPHER = des_enc_list(TEST_KEY_BYTES, TEST_MSG)



    p = 1
    avg = [2**15 for _ in range(3)]
    avg.append(128)
    for x in np.abs(np.array(avg) - np.array(TEST_KEY_GROUP)).tolist():
        p *= x

    exp_mean.append(log2(p))
    print(log2(p))

    p = 1
    for i in range(4):
        err = abs(round(TEST_MSG, TEST_CIPHER, TEST_KEY_GROUP, i, N_SAMPLES))
        p *= err

    exp_us.append(log2(p))
    print(log2(p))

49.416114700621314
45.47289411801147
49.95950779260544
42.32318815700669
